In [73]:

# libraries

import pandas as pd
from ipyvizzu import Chart, Data, Config, Style
from ipyvizzustory import Story, Slide, Step


## Aggregate demographics

In [125]:
# read old pop data

pop_old = pd.read_csv('./data/uk_adult_pop.csv')
pop_old['sexual_orientation'] = "Unknown"

# read 2021 data
pop_2021 = pd.read_excel('data\msoa_pop_by_orientation.xlsx')[['Sexual orientation (6 categories)', 'Observation']]

# cleaning
pop_2021 = pop_2021.rename(columns = {'Sexual orientation (6 categories)' : 'sexual_orientation', 'Observation' : 'population'})
pop_2021 = pop_2021.groupby(['sexual_orientation']).sum().reset_index()
pop_2021 = pop_2021[pop_2021['sexual_orientation'] != 'Does not apply']
pop_2021 = pop_2021.replace('All other sexual orientations', 'Other')
pop_2021['year'] = 2021

# append
pop = pop_old._append(pop_2021).sort_values('year')
pop['year'] = pop['year'].astype(str)
# pop['Year'] = pop['Year'].apply(lambda x: '01-01-' + str(x))
# pop['Year'] = pd.to_datetime(pop['Year'])

# change units
pop['population'] = pop['population'].apply(lambda x: x/1000000).round(2)
pop = pop.rename(columns = {'population' : 'Population (M)',
                            'sexual_orientation' : 'Sexual Orientation',
                            'year' : 'Year'})

pop



,Year,Population (M),Sexual Orientation
5,1961,34.91,Unknown
4,1971,36.82,Unknown
3,1981,38.72,Unknown
2,1991,40.50,Unknown
1,2001,41.86,Unknown
0,2011,45.58,Unknown
0,2021,0.17,Other
1,2021,0.62,Bisexual
3,2021,0.75,Gay or Lesbian
4,2021,3.63,Not answered


In [131]:
data = Data()
data.add_df(pop)

# chart

story = Story(data)

slide1 = Slide(
    Step(
        Data.filter("record.Year != '2021'"),
        Config.stackedColumn({"x": "Year",
                "y": "Population (M)",
                "title": "Adult (16+) population in England and Wales",
                "stackedBy": "Sexual Orientation"
                })
    )
)
story.add_slide(slide1)

slide2 = Slide()

slide2.add_step(
    Step(
        Data.filter("record.Year <= '2021'")
))

slide2.add_step(
    Step(
        Data.filter("record.Year == '2021'"),
        Config({
            'y' : 'Sexual Orientation',
            'x' : 'Population (M)',
            'coordSystem' : 'polar',
            'title': '2021 breakdown of adult sexual orientation in England and Wales'
                })
    )
)

story.add_slide(slide2)



story.play()